In [1]:
#This ipython notebook used to find the outliters and replace the outliers

In [2]:
import pandas as pd #to read the dataset pandas lib will be used

In [3]:
dataset=pd.read_csv("Placement_Data_Full_Class.csv") #reading the placement dataset

In [4]:
dataset

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0
211,212,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0
212,213,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0
213,214,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0


In [6]:
#Creating empty list for Quantiative and Qualitative
Qual=[]
Quan=[]

In [7]:
# Separating Quan and Qual columns

for columnName in dataset.columns:
    #print(columnName)
    if(dataset[columnName].dtypes == 'O'):
        #print("Qual")
        Qual.append(columnName)
    else:
        #print("Quan")
        Quan.append(columnName)  

# IQR - Inter Quartile Range


In [8]:
MLD=pd.DataFrame(index=["Q1:25%","Q3:75%","IQR","1.5Rule","Lesser Outlier","Greater Outlier","Minimum","Maximum"],
                 columns=Quan) # Table Creation with proper row and columns

In [9]:
for columnName in Quan:
    MLD[columnName]["Q1:25%"]= dataset.describe()[columnName]["25%"]
    MLD[columnName]["Q3:75%"]= dataset.describe()[columnName]["75%"]
    
    #From here IQR, 1.5rule, Lesser Outlier and Greater Outlier coding starts. 
    
    #IQR = Q3-Q1
    MLD[columnName]["IQR"]=MLD[columnName]["Q3:75%"]-MLD[columnName]["Q1:25%"]
    
    #1.5Rule = 1.5*IQR
    MLD[columnName]["1.5Rule"]=1.5*MLD[columnName]["IQR"]
    
    #Lesser Outlier = Q1-1.5*IQR
    MLD[columnName]["Lesser Outlier"]=MLD[columnName]["Q1:25%"]-MLD[columnName]["1.5Rule"]
    
    #Greater Outlier = Q3+1.5*IQR
    MLD[columnName]["Greater Outlier"]=MLD[columnName]["Q3:75%"]+MLD[columnName]["1.5Rule"]
    
    #Minimum
    MLD[columnName]["Minimum"]=dataset[columnName].min()
    
    #Maximum
    MLD[columnName]["Maximum"]=dataset[columnName].max()

In [10]:
MLD

,sl_no,ssc_p,hsc_p,degree_p,etest_p,mba_p,salary
Q1:25%,54.5,60.6,60.9,61,60,57.945,240000
Q3:75%,161.5,75.7,73,72,83.5,66.255,300000
IQR,107,15.1,12.1,11,23.5,8.31,60000
1.5Rule,160.5,22.65,18.15,16.5,35.25,12.465,90000
Lesser Outlier,-106,37.95,42.75,44.5,24.75,45.48,150000
Greater Outlier,322,98.35,91.15,88.5,118.75,78.72,390000
Minimum,1,40.89,37,50,50,51.21,200000
Maximum,215,89.4,97.7,91,98,77.89,940000


In [11]:
#splitting lesser outlier columns and greater outlier columns in two different lists
lesser=[]
greater=[]

# Finding Outlier Columns

In [12]:
for columnName in Quan:
    if MLD[columnName]["Minimum"] < MLD[columnName]["Lesser Outlier"]: #logic to find lesser outlier columns in the dataframe
        lesser.append(columnName)
    if MLD[columnName]["Maximum"] > MLD[columnName]["Greater Outlier"]:  #logic to find greater outlier columns in the dataframe
        greater.append(columnName)

In [13]:
lesser

['hsc_p']

In [14]:
greater

['hsc_p', 'degree_p', 'salary']

# Replacing Outliers

In [15]:
for columnName in lesser:
    dataset[columnName][dataset[columnName]<MLD[columnName]["Lesser Outlier"]]=MLD[columnName]["Lesser Outlier"]
for columnName in greater:
    dataset[columnName][dataset[columnName]>MLD[columnName]["Greater Outlier"]]=MLD[columnName]["Greater Outlier"]    

C:\Users\krish\Anaconda3\envs\krish_virtual\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\krish\Anaconda3\envs\krish_virtual\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [16]:
MLD=pd.DataFrame(index=["Q1:25%","Q3:75%","IQR","1.5Rule","Lesser Outlier","Greater Outlier","Minimum","Maximum"],
                 columns=Quan) # Table Creation with proper row and columns

for columnName in Quan:
    MLD[columnName]["Q1:25%"]= dataset.describe()[columnName]["25%"]
    MLD[columnName]["Q3:75%"]= dataset.describe()[columnName]["75%"]
    
    #From here IQR, 1.5rule, Lesser Outlier and Greater Outlier coding starts. 
    
    #IQR = Q3-Q1
    MLD[columnName]["IQR"]=MLD[columnName]["Q3:75%"]-MLD[columnName]["Q1:25%"]
    
    #1.5Rule = 1.5*IQR
    MLD[columnName]["1.5Rule"]=1.5*MLD[columnName]["IQR"]
    
    #Lesser Outlier = Q1-1.5*IQR
    MLD[columnName]["Lesser Outlier"]=MLD[columnName]["Q1:25%"]-MLD[columnName]["1.5Rule"]
    
    #Greater Outlier = Q3+1.5*IQR
    MLD[columnName]["Greater Outlier"]=MLD[columnName]["Q3:75%"]+MLD[columnName]["1.5Rule"]
    
    #Minimum
    MLD[columnName]["Minimum"]=dataset[columnName].min()
    
    #Maximum
    MLD[columnName]["Maximum"]=dataset[columnName].max()


In [17]:
MLD

,sl_no,ssc_p,hsc_p,degree_p,etest_p,mba_p,salary
Q1:25%,54.5,60.6,60.9,61,60,57.945,240000
Q3:75%,161.5,75.7,73,72,83.5,66.255,300000
IQR,107,15.1,12.1,11,23.5,8.31,60000
1.5Rule,160.5,22.65,18.15,16.5,35.25,12.465,90000
Lesser Outlier,-106,37.95,42.75,44.5,24.75,45.48,150000
Greater Outlier,322,98.35,91.15,88.5,118.75,78.72,390000
Minimum,1,40.89,42.75,50,50,51.21,200000
Maximum,215,89.4,91.15,88.5,98,77.89,390000


In [24]:
# FINDING OUTLIERS
lessernew=[]
greaternew=[]

In [25]:
for columnName in Quan:
    if MLD[columnName]["Minimum"] < MLD[columnName]["Lesser Outlier"]: #logic to find lesser outlier columns in the dataframe
        lessernew.append(columnName)
    if MLD[columnName]["Maximum"] > MLD[columnName]["Greater Outlier"]:  #logic to find greater outlier columns in the dataframe
        greaternew.append(columnName)

In [26]:
lessernew

[]

In [27]:
greaternew

[]